In [1]:
import pandas as pd
import os
import numpy as np
import datetime
import gc
os.getcwd()



'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327'

In [2]:
samplerows=10

def recrusive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root_path,file)
            

In [3]:
list_2018Q4_sales=list(recrusive_file_gen("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"))
list_2018Q4_sales=[x for x in list_2018Q4_sales if ".txt" in x]
df_2018Q4_sales_files=pd.DataFrame({"file_path":list_2018Q4_sales})
df_2018Q4_sales_files['week_end_dt']=df_2018Q4_sales_files['file_path'].apply(lambda x: x.split("ompressed/MediaStormDailySalesHistory")[1][:8])
df_2018Q4_sales_files=df_2018Q4_sales_files[(df_2018Q4_sales_files['week_end_dt']>="20181110") & (df_2018Q4_sales_files['week_end_dt']<="20190202")]
df_2018Q4_sales_files.shape

(13, 2)

In [4]:
product_taxonomy=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190501-135142-836.txt",dtype=str,sep="|")
product_taxonomy=product_taxonomy[['division_id','class_code_id','subclass_id']].drop_duplicates()
division_table=product_taxonomy.copy()

divison_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",dtype=str,sep="|")
product_taxonomy=pd.merge(product_taxonomy,divison_name,on="division_id",how="left")

In [5]:
samplerows=None
df_2018Q4_sales_all=pd.DataFrame()
df_2018Q4_sales_div=pd.DataFrame()
i_counter=0
for file in df_2018Q4_sales_files['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,nrows=samplerows,usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','class_code_id','subclass_id','item_transaction_units','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    # Negative sales included in the transction count
    
    df_trans_all=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans_all=df_trans_all.groupby(['location_id','transaction_dt','customer_id_hashed'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    
    df_sales_all=df.groupby(['location_id','transaction_dt','customer_id_hashed'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales","item_transaction_units":"units"})
    df_all=pd.merge(df_sales_all,df_trans_all,on=['location_id','transaction_dt','customer_id_hashed'])
    
    df_2018Q4_sales_all=df_2018Q4_sales_all.append(df)
  
    
    # division level
    df=pd.merge(df,division_table,on=['class_code_id','subclass_id'],how="left")
    df_trans_div=df[['location_id','transaction_dt','transaction_id','customer_id_hashed','division_id']].drop_duplicates()
    df_trans_div=df_trans_div.groupby(['location_id','transaction_dt','customer_id_hashed','division_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    
    df_sales_div=df.groupby(['location_id','transaction_dt','customer_id_hashed','division_id'])['item_transaction_amt','item_transaction_units'].sum().reset_index().rename(columns={"item_transaction_amt":"sales",'item_transaction_units':'units'})
    df_div=pd.merge(df_sales_div,df_trans_div,on=['location_id','transaction_dt','customer_id_hashed','division_id'])
    
    df_2018Q4_sales_div=df_2018Q4_sales_div.append(df_div)
    i_counter+=1
    print(datetime.datetime.now(),i_counter,file)
    
del df
del df_all
del df_div
gc.collect()

2019-05-06 09:36:55.805998 1 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt
2019-05-06 09:40:48.293979 2 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt
2019-05-06 09:45:32.784957 3 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181124.txt
2019-05-06 09:49:38.651399 4 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181201.txt
2019-05-06 09:55:21.253334 5 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181208.txt
2019-05-06 10:01:23.155297 6 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181215.txt
2019-05-06 10:11:07.742324 7 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181222.txt
2019-05-06 10:16:42.547694 8 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHist

189

In [6]:
print(len(df_2018Q4_sales_all['customer_id_hashed'].unique()))
print(len(df_2018Q4_sales_div['customer_id_hashed'].unique()))
print(df_2018Q4_sales_all.shape)
print(df_2018Q4_sales_div.shape)

df_2018Q4_sales_all.head(2)

9777953
9777953
(135159718, 8)
(48854911, 7)


,location_id,transaction_dt,transaction_id,customer_id_hashed,class_code_id,subclass_id,item_transaction_units,item_transaction_amt
0,4621,2018-11-10,3371,34e22ad0c4e216b4983bc2978770b3dbd7301213f8ebfd...,12006,1,2,3.0
2,4642,2018-11-10,3249,049f7c29a72fd252a880b32912fa49adc5dc55f3047193...,36007,1,1,32.0


In [8]:
df_2018Q4_sales_all[df_2018Q4_sales_all['item_transaction_amt']<0].head(10)

,location_id,transaction_dt,transaction_id,customer_id_hashed,class_code_id,subclass_id,item_transaction_units,item_transaction_amt
89,5229,2018-11-10,932,3f2d8d59c8000e49efaaaf1e87d9bac7e041031be0af9d...,11003,3,-1,-19.99
654,1973,2018-11-08,3227,f548049f2d5702ab3b81a00b17dd3d4e05fdd9ac7a12af...,55011,3,-1,-10.00
924,4564,2018-11-05,1076,316c4138ad09749874e9a63757d34ceae1ba65bee039e2...,35067,1,-1,-6.00
933,1118,2018-11-05,7170,b12e7aaf27ec5cc483515455e3ede18438010354a2d2f8...,36002,1,-1,-12.00
1131,514,2018-11-10,161,89741c47bb6171d44675679430a35fa1aafd9c4c8d4df2...,36009,7,-1,-4.00
1447,5202,2018-11-08,1825,cb9495519af8167217a60ee2e4a0d26fecc27e3400e047...,35050,4,-1,-10.00
1946,5316,2018-11-09,4616,f7f0e9819f533d2e1c83491a6704aa78a6b03b6e3bdae7...,17001,6,-2,-15.00
1984,1570,2018-11-08,5203,1a8f8b19fc103f69e53eccad9f6fc60b160f93e1554c34...,35073,1,-1,-6.00
2016,1022,2018-11-10,4155,e288b46b19663df2bb309274b0c9cbaa0eda88c250c6bf...,36002,1,-2,-24.00
2142,1950,2018-11-06,8282,805c7948a183745800bdddb65716976a4a0f43478e3be4...,16003,4,-1,-10.00


In [9]:
os.getcwd()
df_2018Q4_sales_all.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327/Q4_rewards_sales_trans.csv",index=False)
del df_2018Q4_sales_all
gc.collect()

189

In [10]:
'''
print(df_2018Q4_sales_all['transaction_dt'].min(),df_2018Q4_sales_all['transaction_dt'].max())
print(df_2018Q4_sales_div['transaction_dt'].min(),df_2018Q4_sales_div['transaction_dt'].max())

2018-11-04 2019-02-02
2018-11-04 2019-02-02
'''

"\nprint(df_2018Q4_sales_all['transaction_dt'].min(),df_2018Q4_sales_all['transaction_dt'].max())\nprint(df_2018Q4_sales_div['transaction_dt'].min(),df_2018Q4_sales_div['transaction_dt'].max())\n\n2018-11-04 2019-02-02\n2018-11-04 2019-02-02\n"

In [14]:
df=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327/BL_Q4_rewards_migration_rewards_data_JL_2019-03-30.csv",
           dtype=str)
df['sales_2018_Q4']=df['sales_2018_Q4'].astype(float)
df_Q4_shopped=df[df['Q4_Shopping_Group']=="Q4_Shopped"]

print(len(df_Q4_shopped['customer_id_hashed'].unique()))
print(df_Q4_shopped['Group_after_Q4'].unique())
df_Q4_shopped=df_Q4_shopped[['customer_id_hashed','Group_before_Q4','Group_after_Q4','sales_2018_Q4','NewRewards']]


9777953
['Active']


In [23]:
df_2018Q4_sales_div=pd.merge(df_2018Q4_sales_div,df_Q4_shopped,on="customer_id_hashed",how="left")
del df_Q4_shopped
gc.collect()

59

In [25]:
df_2018Q4_sales_div.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Compare_Q4_Before_After_Groups_JT_20190327/BL_Q4_rewards_div_groups_data_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

